In [1]:
#https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=OvvBAoQVJsuU

from collections import deque
import random
import math
import time
from tqdm import tqdm

import numpy as np
import tensorflow as tf
import gym
import gym_examples


In [2]:
'''
env = gym.make('CartPole-v1')
num_features = env.observation_space.shape[0]
num_actions = env.action_space.n
print('Number of state features: {}'.format(num_features))
print('Number of possible actions: {}'.format(num_actions))
'''

"\nenv = gym.make('CartPole-v1')\nnum_features = env.observation_space.shape[0]\nnum_actions = env.action_space.n\nprint('Number of state features: {}'.format(num_features))\nprint('Number of possible actions: {}'.format(num_actions))\n"

In [3]:

env = gym.make('gym_examples/PuckWorld-v0', reward1=True, reward2=True, render_mode=None, fps=60)  #'human'
env._max_episode_steps = 600
#num_actions = 5 #PuckWorld: 0,1,2,3,4 represent left, right, up, down, -, five moves.
num_features = env.observation_space.shape[0]
num_actions = env.action_space.n
print('Number of state features: {}'.format(num_features))
print('Number of possible actions: {}'.format(num_actions))

Number of state features: 8
Number of possible actions: 5


C:\Users\remyh\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
class DQN(tf.keras.Model):
    """Dense neural network class."""
    def __init__(self):
        super(DQN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(32, activation="relu")
        self.dense3 = tf.keras.layers.Dense(num_actions, dtype=tf.float32) # No activation
    
    def call(self, x):
        """Forward pass."""
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

main_nn = DQN()
target_nn = DQN()

optimizer = tf.keras.optimizers.Adam(1e-4)
mse = tf.keras.losses.MeanSquaredError()

In [5]:
class ReplayBuffer(object):
    """Experience replay buffer that samples uniformly."""
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def __len__(self):
        return len(self.buffer)

    def sample(self, num_samples):
        states, actions, rewards, next_states, dones = [], [], [], [], []
        idx = np.random.choice(len(self.buffer), num_samples)
        for i in idx:
            elem = self.buffer[i]
            state, action, reward, next_state, done = elem
            states.append(np.array(state, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(reward)
            next_states.append(np.array(next_state, copy=False))
            dones.append(done)
        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards, dtype=np.float32)
        next_states = np.array(next_states)
        dones = np.array(dones, dtype=np.float32)
        return states, actions, rewards, next_states, dones

In [6]:
def select_epsilon_greedy_action(state, epsilon):
    """Take random action with probability epsilon, else take best action."""
    result = tf.random.uniform((1,))
    if result < epsilon:
        return env.action_space.sample() # Random action (left or right).
    else:
        return tf.argmax(main_nn(state)[0]).numpy() # Greedy action for state.

In [7]:
@tf.function
def train_step(states, actions, rewards, next_states, dones):
    """Perform a training iteration on a batch of data sampled from the experience
    replay buffer."""
    # Calculate targets.
    next_qs = target_nn(next_states)
    max_next_qs = tf.reduce_max(next_qs, axis=-1)
    target = rewards + (1. - dones) * discount * max_next_qs
    with tf.GradientTape() as tape:
        qs = main_nn(states)
        action_masks = tf.one_hot(actions, num_actions)
        masked_qs = tf.reduce_sum(action_masks * qs, axis=-1)
        loss = mse(target, masked_qs)
    grads = tape.gradient(loss, main_nn.trainable_variables)
    optimizer.apply_gradients(zip(grads, main_nn.trainable_variables))
    return loss

In [8]:
# Hyperparameters.
num_episodes = 1000
epsilon = 1
batch_size = 32
discount = 0.99
buffer = ReplayBuffer(100000)
cur_frame = 0

In [10]:


# Start training. Play game once and then train with a batch.
last_10_ep_rewards = []
for episode in tqdm(range(num_episodes+1)):
    state, info = env.reset()  
    ep_reward, done, truncated = 0, False, False
    while not (done or truncated):
        state_in = tf.expand_dims(state, axis=0)
        action = select_epsilon_greedy_action(state_in, epsilon)
        next_state, reward, done, truncated, info = env.step(action)
        ep_reward += reward
        # Save to experience replay.
        buffer.add(state, action, reward, next_state, done or truncated)
        state = next_state
        cur_frame += 1
        # Copy main_nn weights to target_nn.
        if cur_frame % 2000 == 0:
            target_nn.set_weights(main_nn.get_weights())

        # Train neural network.
        if len(buffer) >= batch_size:
            states, actions, rewards, next_states, dones = buffer.sample(batch_size)
            loss = train_step(states, actions, rewards, next_states, dones)
  
    if episode < 950:
        epsilon -= 0.001
    if epsilon<0.005:
        epsilon=0.005

    if len(last_10_ep_rewards) == 10:
        last_10_ep_rewards = last_10_ep_rewards[1:]
    last_10_ep_rewards.append(ep_reward)
    
    if episode % 10 == 0:
        print(f'Episode {episode}/{num_episodes}. Epsilon: {epsilon:.3f}. '
            f'Reward in last 10 episodes: {np.mean(last_10_ep_rewards):.3f}')
env.close()


  0%|                                                                                 | 1/1001 [00:02<36:24,  2.18s/it]

Episode 0/1000. Epsilon: 0.049. Reward in last 10 episodes: -253.596


  1%|▉                                                                               | 11/1001 [00:23<35:46,  2.17s/it]

Episode 10/1000. Epsilon: 0.039. Reward in last 10 episodes: -226.114


  2%|█▋                                                                              | 21/1001 [00:45<35:33,  2.18s/it]

...

Episode 980/1000. Epsilon: 0.005. Reward in last 10 episodes: -263.766


 99%|██████████████████████████████████████████████████████████████████████████████▏| 991/1001 [43:04<00:26,  2.68s/it]

Episode 990/1000. Epsilon: 0.005. Reward in last 10 episodes: -242.826


100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [43:32<00:00,  2.61s/it]

Episode 1000/1000. Epsilon: 0.005. Reward in last 10 episodes: -240.968


In [ ]:
envh = gym.make('gym_examples/PuckWorld-v0', reward1=True, reward2=True, render_mode='human', fps=60)  #'human'
env._max_episode_steps = 600
#envh = env = gym.make('CartPole-v1', render_mode='human')
state, info = envh.reset()
done, truncated = False, False
ep_rew = 0
while not done or truncated:
    envh.render()
    state = tf.expand_dims(state, axis=0)
    action = select_epsilon_greedy_action(state, epsilon=0.01)
    state, reward, done, truncated, info = envh.step(action)
    ep_rew += reward
print('Episode reward was {}'.format(ep_rew))
env.close()


In [ ]:
state
